In [1]:
import pandas as pd
import requests
from tqdm import tqdm

In [2]:
filepath = "/Users/vinay_kl/Downloads/AXIO_Integration_Basic_Sanity_Check [Tamil & Telugu] - NER_Report.csv"
language = "tamil"

In [3]:
df = pd.read_csv(filepath)
df.head()

,QA_Person,Language,Session_ID,Utterance,Translation,Issue_Type,Entity,Captured_Entity (inc percentage),Expected_Entity,Blocking Issue,NER_Team_Status,NER Team Comments,Unnamed: 12,[20 April] Retesting Status
0,Manikandan Cithan,Tamil,a7e8719d-11e5-4275-aa46-6e3d5dfc8d15,வெள்ளிக்கிழமை நைட் 10 மணிக்கு மேல கால் பண்ணு ன...,Call me after Night 10 o Clock on Friday,Missed_Entity,Time,number : 10,22:00,No,In-progress,NaN,NaN,NaN
1,Santosh R,Tamil,5a4fd7bc-948c-4d1a-967f-b7a80c1641a5,இன்னைக்கு பணம் செலுத்த முடியும்.,Today I can Pay,Missed_Entity,Date,NaN,today's date,Yes,In-progress,NaN,NaN,NaN
2,Santosh R,Tamil,df2ed73c-62ab-4728-a367-d451dff1caab,இன்னைக்கு பணம் செலுத்த.,Today I can Pay,Missed_Entity,Date,NaN,today's date,Yes,In-progress,NaN,NaN,NaN
3,Manikandan Cithan,Tamil,4ab1903b-6fb2-4539-bf47-b50bf97682da,சண்டே டூ பிளாக்.,Sunday to Block,Missed_Entity,Date,NaN,09/04/23,Yes,In-progress,NaN,NaN,NaN
4,Manikandan Cithan,Tamil,60ad64de-3cd5-40f4-bd87-78581ce08833,இல்ல மேடம் வாய்ப்பு இல்ல நான் என்ன அடுத்த வாரத...,No madam not possible shall i pay after next week,Missed_Entity,Date,NaN,12/04/23,Yes,In-progress,NaN,NaN,NaN


In [6]:
label_values = []
utterances = df["Utterance"]
#language = df["language"]


for i in tqdm(range(len(df))):
    text = utterances[i]
    values = {}
#     response = requests.post('http://52.151.242.140:80/api/v1/service/lead-gen-nlu-maia/score',json={"data": str(text)}).json()
#     for layer in list(response.keys()):
#         values[layer] = response[layer][0]
#     ner_response = requests.post('http://52.151.242.140:80/api/v1/service/testing-ner/score',json={"data": str(text), "lang": language[i]}).json()
    ner_response = requests.post('http://20.204.203.58:8000/predict',json={"data": str(text), "lang": "tamil", "created_at": "26/01/2022 12:00:00"}).json()
    if len(ner_response["entities"]) > 0:
        for entity in ner_response['entities']:
            if entity['entity'] == 'date':
                values['date'] = entity['value']
            if entity['entity'] == 'time':
                values['time'] = entity['value']
        if "tags" in ner_response:
          values['tags'] = ner_response['tags']
    if "tags" not in values:
        values['tags'] = []
    if 'date' not in values:
        values['date'] = []
    if 'time' not in values:
        values['time'] = []
    label_values.append(values)

  0%|                                                   | 0/110 [00:01<?, ?it/s]


KeyError: 'entities'

In [7]:
label_values[1]

{'sub_context': {'name': 'action_by_customer', 'confidence': '0.9957552'},
 'language': {'name': 'hindi', 'confidence': '0.9999995'},
 'intent': {'name': 'inform', 'confidence': '0.9983012'},
 'context': {'name': 'general', 'confidence': '0.99760747'},
 'signal': {'name': 'no_signal', 'confidence': '0.7315316'},
 'third_person': {'name': 'no', 'confidence': '0.9999298'},
 'humiliate': {'name': 'no_humiliation', 'confidence': '0.99999595'},
 'sub_intent': {'name': 'affirm', 'confidence': '0.6906584'},
 'delay_reason': {'name': 'no_delay_reason', 'confidence': '0.98865664'},
 'sentiment': {'name': 'neutral', 'confidence': '0.9999006'},
 'date': '20/02/2023',
 'tags': ['O', 'B-ptp_date', 'L-ptp_date', 'O', 'O', 'O', 'O'],
 'time': []}

In [8]:
for layer in values:
    name, confidence = [], []
    pred_date, pred_time, pred_tags = [], [], []
    for values in label_values:
        if layer != 'date' and \
          layer != 'time' and \
          layer != 'tags':
          name.append(values[layer]['name'])
          confidence.append(values[layer]['confidence'])
        elif layer == 'date' or layer == "time" or layer == "tags":
          pred_date.append(values["date"])
          pred_time.append(values["time"])
          pred_tags.append(values["tags"])

    if layer != 'date' and \
          layer != 'time' and \
          layer != 'tags':
        df[layer] = name
        df[layer+"_conf"] = confidence
    else:
      df["date"] = pred_date
      df["time"] = pred_time
      df["tags"] = pred_tags

In [9]:
df.head()

,Combinations,language,English,Hindi,Tamil,Telugu,Kannada,Malayalam,Marathi,Bangla,...,third_person,third_person_conf,humiliate,humiliate_conf,sub_intent,sub_intent_conf,delay_reason,delay_reason_conf,sentiment,sentiment_conf
0,Weekdays,hindi,I will make the payment on this Sunday,मैं इस संडे को पे करूंगा,இந்த ஞாயித்துக்கிழம பணம் கொடுக்குரன்,నేను ఈ ఆదివారం చెల్లింపు చేస్తాను,ಈ ಭಾನುವಾರದಂದು ನಾನು ಪಾವತಿ ಮಾಡುತ್ತೇನೆ,ഞാൻ ഈ ഞായറാഴ്ച പേയ്മെന്റ് നടത്തും,मी या रविवारी पेमेंट देईन,আমি এই রবিবার পেমেন্ট করব,...,no,0.99986887,no_humiliation,0.99999833,affirm,0.95855945,no_delay_reason,0.9991528,neutral,0.9999722
1,Weekdays,hindi,"By Monday, i can do the payment",मैं मंडे तक पे कर सकता हूँ,திங்ககிழமைக்குள்ள பேமண்ட் பண்ணிரன்,"సోమవారం నాటికి, నేను చెల్లింపు చేయగలను","ಸೋಮವಾರದ ವೇಳೆಗೆ, ನಾನು ಪಾವತಿ ಮಾಡಬಹುದು",തിങ്കളാഴ്ചയോടെ എനിക്ക് പേയ്മെന്റ് ചെയ്യാൻ കഴിയും,सोमवारपर्यंत मी पेमेंट देऊ शकतो,সোমবারের মধ্যে আমি পেমেন্ট করতে পারি,...,no,0.9999298,no_humiliation,0.99999595,affirm,0.6906584,no_delay_reason,0.98865664,neutral,0.9999006
2,Weekdays,hindi,"ahh this Tuesday, i will do the payment",अहह मैं इस ट्यूसडे को पे करूंगा,வர செவ்வாக்கிழம நான் பணம் கொடுக்குரன்,"ఈ మంగళవారం ఆహ్, నేను చెల్లింపు చేస్తాను","ಆಹ್ ಈ ಮಂಗಳವಾರ , ನಾನು ಪಾವತಿ ಮಾಡುತ್ತೇನೆ",ഈ ചൊവ്വാഴ്ച ഞാൻ പേയ്മെന്റ് ചെയ്യും,या मंगळवारी मी पेमेंट देईन,আহ এই মঙ্গলবার আমি পেমেন্ট করব,...,no,0.9996983,no_humiliation,0.9999989,affirm,0.95684963,no_delay_reason,0.9968945,neutral,0.9999821
3,Weekdays,hindi,I will do on Wednesday,मैं वेडनेसडे को करूंगा,நான் புதன்கிழமை தரன்,నేను బుధవారం చేస్తాను,ನಾನು ಬುಧವಾರ ಮಾಡುತ್ತೇನೆ,ഞാൻ ബുധനാഴ്ച ചെയ്യും,मी बुधवारी करेन,আমি বুধবার করব,...,no,0.97225183,no_humiliation,0.9999442,affirm,0.5784244,no_delay_reason,0.88670605,neutral,0.9995061
4,Weekdays,hindi,Thursday Thursday,थर्सडे थर्सडे,வியாழக்கிழமை டியூஸ்டே,గురువారం గురువారం,ಗುರುವಾರ ಗುರುವಾರ,വ്യാഴാഴ്ച വ്യാഴാഴ്ച,गुरुवारी गुरुवार,বৃহস্পতিবার বৃহস্পতিবার,...,no,0.99999404,no_humiliation,0.9997985,general_ask_inform,0.9876144,no_delay_reason,0.99245137,neutral,0.99738115


In [10]:
df.to_csv("/Users/vinay_kl/Downloads/NER_HI_14Feb.csv", index = False)